# GBR Evaluation

In [1]:
from google.colab import drive
drive.mount('/content/drive')
file = r'/content/drive/MyDrive/Colab/gbr-mcr/dataset.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(file)

# dataset
dataset = df.values
X = dataset[:, 6] # mt column
Y = dataset[:, 5] # mcr column

In [3]:
from sklearn.model_selection import train_test_split
X = X.reshape(-1, 1)

# split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# size
print("Train: ", X_train.shape)
print("Val: ", X_val.shape)
print("Test: ", X_test.shape)

Train:  (115196, 1)
Val:  (28800, 1)
Test:  (36000, 1)


In [4]:
from sklearn.ensemble import GradientBoostingRegressor

# model
gbr = GradientBoostingRegressor(subsample=0.8,
                                learning_rate=0.02,
                                n_estimators=300,
                                max_depth=8,
                                min_samples_split=200, 
                                min_samples_leaf=40,
                                loss="absolute_error")

gbr.fit(X_train, Y_train)
gbr.score(X_val, Y_val)

0.6206419137924768